In [1]:
from dask.distributed import Client, as_completed

In [2]:
# Import/Define all the things the workers will need before starting the cluster
import numpy as np
import time
def run_fake_md(ens, input_traj, output_traj, n=100, offset=1, *args, **kwargs):
    # all of these have to be list for possible more than 1 result, such as with 0+- swap
    out = {'ensembles': [],
           'accepted_trajs': [],
           'traj_vectors': [],
           'r_times': []}
    for e, it, ot in zip(ens, input_traj, output_traj):
        traj = fake_traj(e, n, offset)
        # sleep for a random time
        r_time = np.random.random()*abs(e)*(10/n) + .1
        time.sleep(r_time)
        out['ensembles'].append(e)
        out['accepted_trajs'].append(ot)
        out['traj_vectors'].append(traj) # here the state vector is identical to the trajectory, not true for other sytems
        out['r_times'].append(r_time)
    return out

def fake_traj(ens=0, n=100, offset=1):
    idx = abs(ens)
    if ens < 0:
        n = offset
    r = np.random.random()
    weight = 1
    l = [weight for i in range(idx)] #always valid up to ens
    l += [int(r<0.1**i)*weight for i in range(n-(idx))]
    if ens < 0: # Minus ensemble
        l.reverse()
    return l

In [3]:
# Start the Client and cluster with n workers
n_workers = 10
size = 51
client = Client(n_workers=n_workers)
client

2022-06-16 11:49:14,547 - distributed.diskutils - INFO - Found stale lock file and directory '/home/sander/waitless/dask_example/dask-worker-space/worker-pt4n2ff0', purging
2022-06-16 11:49:14,547 - distributed.diskutils - INFO - Found stale lock file and directory '/home/sander/waitless/dask_example/dask-worker-space/worker-vtoc2ida', purging
2022-06-16 11:49:14,548 - distributed.diskutils - INFO - Found stale lock file and directory '/home/sander/waitless/dask_example/dask-worker-space/worker-uucvpxkw', purging
2022-06-16 11:49:14,548 - distributed.diskutils - INFO - Found stale lock file and directory '/home/sander/waitless/dask_example/dask-worker-space/worker-xu98jvvo', purging
2022-06-16 11:49:14,562 - distributed.diskutils - INFO - Found stale lock file and directory '/home/sander/waitless/dask_example/dask-worker-space/worker-6e0_kb7b', purging
2022-06-16 11:49:14,564 - distributed.diskutils - INFO - Found stale lock file and directory '/home/sander/waitless/dask_example/dask-w

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 20,Total memory: 15.26 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42305,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 15.26 GiB
Comm: tcp://127.0.0.1:43383,Total threads: 2
Dashboard: http://127.0.0.1:33139/status,Memory: 1.53 GiB
Nanny: tcp://127.0.0.1:39917,


In [4]:
futures = as_completed(None, with_results=True) # This is an iterator that gives back the futures as they are completed
for i in range(10):
    j = client.submit(run_fake_md, [i], ['input_traj'], ['output_traj'], size, offset=0, pure=False)
    futures.add(j) # This is how we add stuff to the iterator


In [5]:
for i in futures:
    print(i[1]['ensembles'])

[0]
[3]
[1]
[5]
[2]
[4]
[9]
[7]
[6]
[8]


In [6]:
futures

<as_completed: waiting=0 done=0>

In [7]:
# Define some infRETIS infrastructure
def calculate_ensemble_prob_and_weight(ens, num, offset=0):
    keys = np.array(list(ens.keys()))
    values = np.array(list(ens.values()))
    results = np.zeros(len(keys[0]), dtype="float128")
    idx = np.argsort(values)
    for key, value in zip(keys[idx], values[idx]):
        # 1/key[num] in the following line is the MC reweighting
        results += (np.array([1 if i else 0 for i in key]) *
                    value/abs(key[num+offset]))
    if num < 0:
        offset -= 1
    if results[offset] == 0:
        raise ValueError(f"{ens},{num},{offset},{results}")

    return results/results[offset], results[offset]


def analyze_data(data, offset=0):
    total_prob = np.array([1.0], dtype="float128")
    overall_prob = [1.0]
    ensembles = np.sort(list(data.data.keys()))
    for ens in ensembles:
        if ens < 0:
            continue
        prob, _ = calculate_ensemble_prob_and_weight(data.data[ens], ens,
                                                     offset=offset)
        if ens+offset+1 >= len(prob):
            total_prob = 0
            overall_prob.append(0)
            break
        next_prob = prob[ens+offset+1]
        overall_prob.append(next_prob*total_prob[0])
        total_prob *= next_prob  # cross next interface
    return overall_prob, total_prob


class Results(object):
    def __init__(self, n=3, offset=0):
        self.data = {}
        self.avg_lengths = {}
        self._run_prob = {}
        self._run_weight = {}
        self._cycles = {}
        self._run_total_prob = []
        self._n = n
        self._offset = offset

    def update_ens(self, ens, traj, weight, length=None):
        e = self.data.get(ens, {})
        w = e.get(traj, 0)
        e[traj] = w+weight
        self.data[ens] = e
        if length is not None and weight != 0:
            # update pathlengths for flux calculation
            avg, total_weight = self.avg_lengths.get(ens, (0, 0))
            total_weight += weight
            avg += weight*(length-avg)/total_weight
            self.avg_lengths[ens] = (avg, total_weight)

    def update_run_prob(self, ens, n=0):
        run_p = self._run_prob.get(ens, [])
        run_w = self._run_weight.get(ens, [])
        run_c = self._cycles.get(ens, [])
        data = self.data.get(ens, None)
        if data is not None:
            temp = calculate_ensemble_prob_and_weight(data, ens,
                                                      offset=self._offset)

            prob, tot_weight = temp
            run_p.append(prob[ens+self._offset+1])
            run_w.append(tot_weight)
            run_c.append(n)
        self._run_prob[ens] = run_p
        self._run_weight[ens] = run_w
        self._cycles[ens] = run_c

    def update_run_total_prob(self):
        _, tot = analyze_data(self, self._offset)
        self._run_total_prob.append(tot)


class REPEX_state(object):
    def __init__(self, n=3, result=None, minus=False):
        if minus:
            self._offset = int(minus)
            n += int(minus)
        else:
            self._offset = 0

        self.n = n
        self.state = np.zeros(shape=(n, n))
        self._locks = np.ones(shape=(n))
        self._last_prob = None
        self._random_count = 0
        self.result = Results(n, offset=self._offset)
        self._n = 0
        self._trajs = ["" for i in range(n)]

    def pick(self):
        prob = self.prob.astype("float64").flatten()
        p = np.random.choice(self.n**2, p=np.nan_to_num(prob/np.sum(prob)))
        traj, ens = np.divmod(p, self.n)
        self.swap(traj, ens)
        self.lock(ens)
        traj = self._trajs[ens]
        # If available do 0+- swap with 50% probability
        if ((
             (ens == self._offset and not self._locks[self._offset-1]) or
             (ens == self._offset-1 and not self._locks[self._offset])
        ) and np.random.random() < 0.5):
            if ens == self._offset:
                # ens = 0
                other = self._offset - 1
                other_traj = self.pick_traj_ens(other)
                return (-1, 0), (other_traj, traj)
            else:
                # ens = -1
                other = self._offset
                other_traj = self.pick_traj_ens(other)
                return (-1, 0), (traj, other_traj)
        return (ens-self._offset,), (traj,)

    def pick_traj_ens(self, ens):
        prob = self.prob.astype("float64")[:, ens].flatten()
        traj = np.random.choice(self.n, p=np.nan_to_num(prob/np.sum(prob)))
        self.swap(traj, ens)
        self.lock(ens)
        return self._trajs[ens]

    def write_ensembles(self):
        out = self.prob
        out = out.T
        out = np.nan_to_num(out)
        for i, ens in enumerate(out):
            if self._locks[i]:
                continue
            for j, weight in enumerate(ens):
                if weight != 0:
                    traj = self.state[j]
                    path = self._trajs[j]
                    if i in [self._offset, self._offset-1]:
                        # ens 0 and -1
                        length = getattr(path, 'length', None)
                    else:
                        length = None
                    self.result.update_ens(i-self._offset, tuple(traj), weight,
                                           length=length)
            self.result.update_run_prob(i-self._offset, n=self._n)
        self.result.update_run_total_prob()

    def add_traj(self, ens, traj, valid, count=True, n=0):
        if ens >= 0 and self._offset != 0:
            valid = tuple([0 for _ in range(self._offset)] + list(valid))
        elif ens < 0:
            valid = tuple(list(valid) +
                          [0 for _ in range(self.n - self._offset)])
        ens += self._offset
        assert valid[ens] != 0
        # invalidate last prob
        self._last_prob = None
        self._trajs[ens] = traj
        self.state[ens, :] = valid
        self.unlock(ens)
        if count:
            self.write_ensembles()
            self._n += 1

    def lock(self, ens):
        # invalidate last prob
        self._last_prob = None
        assert self._locks[ens] == 0
        self._locks[ens] = 1

    def unlock(self, ens):
        # invalidate last prob
        self._last_prob = None
        assert self._locks[ens] == 1
        self._locks[ens] = 0

    def swap(self, traj, ens):
        # mainly to keep the locks symmetric
        self.state[[ens, traj]] = self.state[[traj, ens]].copy()
        temp1 = self._trajs[ens]
        self._trajs[ens] = self._trajs[traj]
        self._trajs[traj] = temp1

    @property
    def prob(self):
        if self._last_prob is None:
            prob = self.inf_retis(abs(self.state), self._locks)
            self._last_prob = prob.copy()
        return self._last_prob

    def inf_retis(self, input_mat, locks):
        # Drop locked rows and columns
        bool_locks = locks == 1
        # get non_locked minus interfaces
        offset = self._offset - sum(bool_locks[:self._offset])
        # make insert list
        i = 0
        insert_list = []
        for lock in bool_locks:
            if lock:
                insert_list.append(i)
            else:
                i += 1

        # Drop locked rows and columns
        non_locked = input_mat[~bool_locks, :][:, ~bool_locks]

        # Sort based on the index of the last non-zero values in the rows
        # argmax(a>0) gives back the first column index that is nonzero
        # so looping over the columns backwards and multiplying by -1 gives the
        # right ordering
        minus_idx = np.argsort(np.argmax(non_locked[:offset] > 0, axis=1))
        pos_idx = (np.argsort(-1 *
                              np.argmax(non_locked[offset:, ::-1] > 0, axis=1)
                              )
                   + offset)

        sort_idx = np.append(minus_idx, pos_idx)
        sorted_non_locked = non_locked[sort_idx]

        # check if all trajectories have equal weights
        sorted_non_locked_T = sorted_non_locked.T
        # Check the minus interfaces
        equal_minus = np.all(sorted_non_locked_T[
            np.where(sorted_non_locked_T[:, :offset] !=
                     sorted_non_locked_T[offset-1, :offset])
            ] == 0)
        # check the positive interfaces
        if len(sorted_non_locked_T) <= offset:
            equal_pos = True
        else:
            equal_pos = np.all(sorted_non_locked_T[:, offset:][
                np.where(sorted_non_locked_T[:, offset:] !=
                         sorted_non_locked_T[offset, offset:])
                ] == 0)

        equal = equal_minus and equal_pos

        out = np.zeros(shape=sorted_non_locked.shape, dtype="float128")
        if equal:
            # All trajectories have equal weights, run fast algorithm
            # run_fast
            # minus move should be run backwards
            out[:offset, ::-1] = self.quick_prob(sorted_non_locked[:offset,
                                                                   ::-1])
            if offset < len(out):
                # Catch only minus ens available
                out[offset:] = self.quick_prob(sorted_non_locked[offset:])
        else:
            #TODO DEBUG print
            print("DEBUG this should not happen outside of wirefencing")
            blocks = self.find_blocks(sorted_non_locked, offset=offset)
            for start, stop, direction in blocks:
                if direction == -1:
                    cstart, cstop = stop-1, start-1
                    if cstop < 0:
                        cstop = None
                else:
                    cstart, cstop = start, stop
                subarr = sorted_non_locked[start:stop, cstart:cstop:direction]
                subarr_T = subarr.T
                if len(subarr) == 1:
                    out[start:stop, start:stop] = 1
                elif np.all(subarr_T[np.where(subarr_T != subarr_T[0])] == 0):
                    # Either the same weight as the last one or zero
                    temp = self.quick_prob(subarr)
                    out[start:stop, cstart:cstop:direction] = temp
                elif len(subarr) <= 12:
                    # We can run this subsecond
                    temp = self.permanent_prob(subarr)
                    out[start:stop, cstart:cstop:direction] = temp
                else:
                    self._random_count += 1
                    #TODO DEBUG PRINTS
                    print(f"random #{self._random_count}, "
                          f"dims = {len(subarr)}")
                    # do n random parrallel samples
                    temp = self.random_prob(subarr)
                    out[start:stop, cstart:cstop:direction] = temp

        out[sort_idx] = out.copy()  # COPY REQUIRED TO NOT BRAKE STATE!!!

        # Make sure we have a valid probability square
        assert np.allclose(np.sum(out, axis=1), 1)
        assert np.allclose(np.sum(out, axis=0), 1)

        # reinsert zeroes for the locked ensembles
        final_out_rows = np.insert(out, insert_list, 0, axis=0)

        # reinsert zeroes for the locked trajectories
        final_out = np.insert(final_out_rows, insert_list, 0, axis=1)

        return final_out

    def find_blocks(self, arr, offset):
        if len(arr) == 1:
            return ((0, 1, 1))
        # Assume no zeroes on the diagonal or lower triangle
        temp_arr = arr.copy()
        # for counting minus blocks
        temp_arr[:offset, :offset] = arr[:offset, :offset].T
        temp_arr[offset:, :offset] = 1  # add ones to the lower triangle
        non_zero = np.count_nonzero(temp_arr, axis=1)
        blocks = []
        start = 0
        for i, e in enumerate(non_zero):
            if e == i + 1:
                direction = -1 if start < offset else 1
                blocks.append((start, e, direction))
                start = e
        return blocks

    def quick_prob(self, arr):
        total_traj_prob = np.ones(shape=arr.shape[0], dtype='float128')
        out_mat = np.zeros(shape=arr.shape, dtype='float128')
        working_mat = np.where(arr != 0, 1, 0)  # convert non-zero numbers to 1

        for i, column in enumerate(working_mat.T[::-1]):
            ens = column*total_traj_prob
            s = ens.sum()
            if s != 0:
                ens /= s
            out_mat[:, -(i+1)] = ens
            total_traj_prob -= ens
            # force negative values to 0
            total_traj_prob[np.where(total_traj_prob < 0)] = 0
        return out_mat

    def force_quick_prob(self, arr):
        # TODO: DEBUG CODE
        # ONLY HERE TO DEBUG THE OTHER MEHTODS
        total_traj_prob = np.ones(shape=arr.shape[0], dtype='float128')
        out_mat = np.zeros(shape=arr.shape, dtype='float128')

        force_arr = arr.copy()
        # Force everything to be identical
        force_arr[np.where(force_arr != 0)] = 1
        for i, column in enumerate(force_arr.T[::-1]):
            ens = column*total_traj_prob
            s = ens.sum()
            if s != 0:
                ens /= s
            out_mat[:, -(i+1)] = ens
            total_traj_prob -= ens
            # force negative values to 0
            total_traj_prob[np.where(total_traj_prob < 0)] = 0
        return out_mat

    def permanent_prob(self, arr):
        out = np.zeros(shape=arr.shape, dtype="float128")
        n = len(arr)
        for i in range(n):
            rows = [r for r in range(n) if r != i]
            sub_arr = arr[rows, :]
            for j in range(n):
                if arr[i][j] == 0:
                    continue
                columns = [r for r in range(n) if r != j]
                M = sub_arr[:, columns]
                f = self.fast_glynn_perm(M)
                out[i][j] = f*arr[i][j]
        return out/max(np.sum(out, axis=1))

    def random_prob(self, arr, n=10_000):
        out = np.eye(len(arr), dtype="float128")
        current_state = np.eye(len(arr))
        choices = len(arr)//2
        even = choices*2 == len(arr)

        # The probability to go right
        prob_right = np.nan_to_num(np.roll(arr, -1, axis=1)/arr)

        # The probability to go left
        prob_left = np.nan_to_num(np.roll(arr, 1, axis=1)/arr)

        start = 0
        zero_one = np.array([0, 1])
        p_m = np.array([1, -1])
        temp = np.where(current_state == 1)

        for i in range(n):
            direction = np.random.choice(p_m)
            if not even:
                start = np.random.choice(zero_one)

            temp_left = prob_left[temp]
            temp_right = prob_right[temp]

            if not even:
                start = np.random.choice(zero_one)

            if direction == -1:
                probs = (temp_left[start:-1:2] *
                         np.roll(temp_right, 1, axis=0)[start:-1:2])
            else:
                probs = temp_right[start:-1:2]*temp_left[start+1::2]

            r_nums = np.random.random(choices)
            success = r_nums < probs

            for j in np.where(success)[0]:
                idx = j*2+start
                temp_state = current_state[:, [idx+direction, idx]]
                current_state[:, [idx, idx+direction]] = temp_state
                temp_state_2 = temp[0][[idx+direction, idx]]
                temp[0][[idx, idx+direction]] = temp_state_2

            out += current_state

        return out/(n+1)

    def fast_glynn_perm(self, M):
        def cmp(a, b):
            if a == b:
                return 0
            elif a > b:
                return 1
            else:
                return -1
        row_comb = np.sum(M, axis=0, dtype='float128')
        n = len(M)

        total = 0
        old_grey = 0
        sign = +1

        binary_power_dict = {2**i: i for i in range(n)}
        num_loops = 2**(n-1)

        for bin_index in range(1, num_loops + 1):
            total += sign * np.multiply.reduce(row_comb)

            new_grey = bin_index ^ (bin_index // 2)
            grey_diff = old_grey ^ new_grey
            grey_diff_index = binary_power_dict[grey_diff]
            direction = 2 * cmp(old_grey, new_grey)
            if direction:
                new_vector = M[grey_diff_index]
                row_comb += new_vector * direction

            sign = -sign
            old_grey = new_grey

        return total//num_loops

    

In [8]:
#start some timing to keep track
walltime = 5*60 #(sec per hour)
size=51
start_time = time.time() 
current_time = start_time

# Setup the infretis simulation
state = REPEX_state(size, minus=False)
offset = state._offset
# Setup system
for i in range(size):
    state.add_traj(ens=i, traj='init_traj', valid=fake_traj(i, size), count=False)
# lock down last ens (as that one alsways enters the state)
state.lock(size-1)

# Define an iterator that is going to be our results queue
futures = as_completed(None, with_results=True)



# Start running
traj_i = 0
for _ in range(n_workers):
    ens, input_traj = state.pick()
    fut = client.submit(run_fake_md, ens, input_traj, f'traj_{traj_i}', n=size, offset=offset, pure=False)
    futures.add(fut)
    traj_i += 1

t_time = 0 # record total MD time (time spend by the workers)
minutes = 0
steps_done = 0
while current_time < start_time+walltime:
    output = next(futures)[1]
    ensembles = output['ensembles']
    out_trajs = output['accepted_trajs']
    traj_vectors = output['traj_vectors']
    r_times = output['r_times']
    for ens, out_traj, traj_v, r_time in zip(ensembles, out_trajs, traj_vectors, r_times):
        t_time +=  r_time
        state.add_traj(ens, out_traj, traj_v) # Record the shooting result
    steps_done += 1
    ens, input_traj = state.pick() # pick a new shooting move
    out_trajs = []
    for _ in ens:
        # here for multi ensemble MD moves
        out_trajs.append(f"traj-{traj_i}")
        traj_i += 1
    fut = client.submit(run_fake_md, ens, input_traj, out_trajs, n=size, offset=offset, pure=False)
    futures.add(fut)
    if current_time-start_time > 60*minutes:
        print(f"steps_done: {steps_done} in {current_time-start_time} seconds")
        minutes = (current_time-start_time)//60 + 1 
    current_time = time.time()


steps_done: 2 in 0.33219480514526367 seconds
steps_done: 258 in 60.03121781349182 seconds
steps_done: 494 in 120.01128101348877 seconds
steps_done: 752 in 180.51051354408264 seconds
steps_done: 1017 in 240.4545931816101 seconds


In [ ]:

client.close()

In [ ]:
def calculate_ensemble_prob(ens):
    results = np.zeros(size, dtype="float128")
    keys = np.array(list(ens.keys()))
    values = np.array(list(ens.values()))
    idx = np.argsort(values)
    for key, value in zip(keys[idx], values[idx]):
        results += abs(np.array(key)*value)
    return results[np.nonzero(results)]/results[0]

def calculate_ensemble_prob_and_weight(ens, num, offset=0):                     
    keys = np.array(list(ens.keys()))                                           
    values = np.array(list(ens.values()))                                       
    results = np.zeros(len(keys[0]), dtype="float128")                          
    idx = np.argsort(values)                                                    
    for key, value in zip(keys[idx], values[idx]):                              
        # 1/key[num] in the following line is the MC reweighting                
        results += np.array([1 if i else 0 for i in key]) * value/abs(key[num+offset])
    return results/results[offset], results[offset]                             


def analyze_data(data, offset=0):                                               
    total_prob = np.array([1.0], dtype="float128")                              
    overall_prob = [1.0]                                                        
    ensembles = np.sort(list(data.data.keys()))                                 
    for ens in ensembles:
        if ens < 0:
            continue
        prob, _ = calculate_ensemble_prob_and_weight(data.data[ens], ens,       
                                            offset=offset)             
        if ens+offset+1 >= len(prob):                                              
            total_prob = 0                                                      
            overall_prob.append(0)                                              
            break                                                               
        next_prob = prob[ens+offset+1]                                          
        overall_prob.append(next_prob*total_prob[0])                            
        total_prob *= next_prob  # cross next interface                         
    return overall_prob, total_prob    


# In[5]:


overall_prob, total_prob = analyze_data(state.result, offset=state._offset)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(overall_prob)
plt.plot([10**-i for i in range(51)])
plt.yscale('log')
plt.ylim(1e-51, 1e1)

In [ ]:
plt.plot(state.result._run_total_prob)
plt.plot([0, len(state.result._run_total_prob)], [1e-50, 1e-50])

In [ ]:
plt.plot([state.result._run_prob[i][-1] for i in range(50)])
plt.plot([0, 49], [0.1, 0.1])